
# BE Meteo 
## HERVIOUX, SOURDEVAL, MANCIET, DUC-MARTIN, LEFAUCONNIER

In [ ]:
path_data <- paste(readLines("path.txt"), collapse = "\n")
path_data

In [ ]:
#install.packages("data.table")
library(data.table)
dataset_meteo <- fread(path_data)
head(dataset_meteo)

# 1 - Description des données
   
Idées :
- Présentation des données (nombre, type (température pression, lieux...))
- Agrégation des valeurs (par ID station météo, par région ?)
- Carte de la France avec lieux des stations
- Carte avec les températures/pression moyenne/min/max des stations
- Données manquantes

In [ ]:
df_meteo <- data.frame(dataset_meteo)
df_meteo <- df_meteo[order(df_meteo$Date), ] # Trier les valeurs par ordre chronologique
head(df_meteo)

In [ ]:
# colnames(df_meteo)

In [ ]:
cat("Nombre de lignes de données :", nrow(df_meteo), "\n")
cat("Nombre de paramètres :", ncol(df_meteo), "\n")
cat("\tdont 14 paramètres décrivant l'emplacement de la station météo\n")
cat("\tdont 19 paramètres relatifs aux nuages/à la nébulosité (+ 4 hauteurs de bases)\n")
cat("\tdont 14 paramètres relatifs à la température\n")
cat("\tdont 7 paramètres relatifs à la pression/barométrie\n")
cat("\tdont 5 paramètres relatifs au vent/rafales\n")
cat("\tdont 5 paramètres relatifs à la pluie (précipitations)\n")
cat("\tdont 3 paramètres relatifs à la neige\n")
cat("\tdont 2 paramètres relatifs à l'humidité/rosée\n")
cat("\t + Date, mois_de_l_annee, Visibilité.horizontale, Temps.présent, Temps.passé.1, Temps.passé.2, Géopotentiel, Etat.du.sol, Phénomène.spécial.1, Phénomène.spécial.2, Phénomène.spécial.3, Phénomène.spécial.4, Temps.passé.1.1, Temps.présent.1")

# TODO : /!\ Erreur 1 paramètre a dû être compté 2 fois (82 colonnes en tout et mon décompte arrive à 83 paramètres)

### Description des stations météo

In [ ]:
# Caractéristiques des stations météo
df_stations <- df_meteo[c('ID.OMM.station', 'Coordonnees', 'Nom', 'Latitude', 'Longitude', 'Altitude', 'communes..name.', 'communes..code.', 'EPCI..name.', 'EPCI..code.', 'department..name.', 'department..code.', 'region..name.', 'region..code.')]
df_unique_stations <- df_stations[!duplicated(df_stations[c('ID.OMM.station')]), ]
cat("Nombre de stations météo :", nrow(df_unique_stations))
head(df_unique_stations[order(df_unique_stations$Nom), ])

### Carte des stations météo

In [ ]:
#install.packages(c("ggplot2", "maps", "ggmap"))
library(ggplot2)
library(maps)
library(ggmap)

options(repr.plot.width = 20, repr.plot.height = 10)

data_points <- data.frame(
  lon = df_unique_stations$Longitude,
  lat = df_unique_stations$Latitude,
  Ville = df_unique_stations$Nom
)

world <- map_data("world")

ggplot() +
  geom_polygon(data = world, aes(x = long, y = lat, group = group),
               fill = "lightgray", color = "white") +
  geom_point(data = data_points, aes(x = lon, y = lat, color = Ville), size = 1) +
  theme_minimal() +
  theme(legend.position = "none") +
  ggtitle("Carte des stations météo")


# Zoom sur l'hexagone  
data_points_metropole <- data_points[(data_points$lat<=52)&(data_points$lat>=42)&(data_points$lon<=10)&(data_points$lon>=-10),]
france <- map_data("france")

ggplot() +
  geom_polygon(data = france, aes(x = long, y = lat, group = group),
               fill = "lightgray", color = "white") +
  geom_point(data = data_points_metropole, aes(x = lon, y = lat, color = Ville), size = 3) +
  theme_minimal() +
  # theme(legend.position = "none") +
  coord_fixed(ratio = 1.2)+
  ggtitle("Carte des stations météo de France métropolitaine")

### Relevé de températures par station

In [ ]:
for (ville in unique(df_meteo$Nom)[1:5]){
  plot(
    df_meteo$Date[df_meteo$Nom==ville],
    df_meteo$`Température`[df_meteo$Nom==ville],
    xlab = "Date",
    ylab = "Température",
    main = ville,
    type = "l",
)}

### Valeurs moyennes des grandeurs mesurées par station

In [ ]:
# install.packages("dplyr")
library(dplyr)

df_grouped <- df_meteo %>%
  group_by(ID.OMM.station) %>%
  summarise(n_mesures = n(),
            T_mean = mean(`Température`,na.rm = TRUE),
            T_min = min(`Température`,na.rm = TRUE),
            T_max = max(`Température`,na.rm = TRUE),
            P_mean = mean(Pression.au.niveau.mer,na.rm = TRUE),
            nebulosite_mean = mean(Nebulosité.totale, na.rm = TRUE)
            )

head(df_grouped)

### Carte des températures extrêmes mesurées par station

In [ ]:
df_station_stats <- merge(x = df_unique_stations, y = df_grouped, by = "ID.OMM.station", all = TRUE)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)

world <- map_data("world")

ggplot() +
  geom_polygon(data = world, aes(x = long, y = lat, group = group),
               fill = "lightgray", color = "white") +
  geom_point(data = df_station_stats, aes(x = Longitude, y = Latitude, color = T_min), size = 1) +
  theme_minimal() +
  # theme(legend.position = "none")+
  ggtitle("Température minimale par station (K)")

ggplot() +
  geom_polygon(data = world, aes(x = long, y = lat, group = group),
               fill = "lightgray", color = "white") +
  geom_point(data = df_station_stats, aes(x = Longitude, y = Latitude, color = T_max), size = 1) +
  theme_minimal() +
  # theme(legend.position = "none")+
  ggtitle("Température maximale par station (K)")

In [ ]:
summary(dataset_meteo)

# 2. Un calcul et étude d'estimateur

Thibault

Température moyenne ou pluie moyenne en fonction des régions

Etude d'un estimateur de la température moyenne sur l'hiver 2019 en France métropolitaine.

In [ ]:
data_points <- data.frame(
  date = df_meteo$Date,
  temp = df_meteo$`Température...C.`,
  station = df_meteo$Nom,
  mois = df_meteo$mois_de_l_annee,
  lat = df_meteo$Latitude,
  long = df_meteo$Longitude
)

france <- map_data("france")

data_points_metropole <- data_points %>%
  filter(lat <= 52, lat >= 42,
         long <= 10, long >= -10,
         date < "2020-01-01", date > "2019-01-01",
         mois > 1, mois < 4) %>%
  group_by(station, lat, long) %>%
  summarise(temp_moy = mean(temp, na.rm = TRUE), .groups = "drop")

ggplot() +
  geom_polygon(data = france, aes(x = long, y = lat, group = group),
               fill = "lightgray", color = "white") +
  geom_point(data = data_points_metropole, aes(x = long, y = lat, color = temp_moy), size = 3) +
  theme_minimal() +
  scale_color_gradient(low = "blue", high = "red",
                     name = "Température moyenne (°C)") +
  coord_fixed(ratio = 1.2)+
  ggtitle("Moyenne des température sur l'hiver 2019")


Estimation des précipitations moyennes en France métropolitaine sur l'hiver 2019

In [ ]:
data_points <- data.frame(
  date = df_meteo$Date,
  pluie = df_meteo$`Précipitations.dans.la.dernière.heure`,
  station = df_meteo$Nom,
  mois = df_meteo$mois_de_l_annee,
  lat = df_meteo$Latitude,
  long = df_meteo$Longitude
)

france <- map_data("france")

data_points_metropole <- data_points %>%
  filter(lat <= 52, lat >= 42,
         long <= 10, long >= -10,
         date < "2020-01-01", date > "2019-01-01",
         mois > 1, mois < 4) %>%
  group_by(station, lat, long) %>%
  summarise(pluie_moy = mean(pluie, na.rm = TRUE), .groups = "drop")

ggplot() +
  geom_polygon(data = france, aes(x = long, y = lat, group = group),
               fill = "lightgray", color = "white") +
  geom_point(data = data_points_metropole, aes(x = long, y = lat, color = pluie_moy), size = 3) +
  theme_minimal() +
  scale_color_gradient(low = "blue", high = "red",
                     name = "Précipitation (mm)") +
  coord_fixed(ratio = 1.2)+
  ggtitle("Moyenne des précipitations sur l'hiver 2019")


In [ ]:
data_points <- data.frame(
  date = df_meteo$Date,
  temp = df_meteo$`Température...C.`,
  lat = df_meteo$Latitude,
  long = df_meteo$Longitude
)

data_points <- data_points%>%
  mutate(annee = year(date))
data_points <- data_points %>%
  group_by(annee) %>%
  summarise(temp_moy = mean(temp, na.rm = TRUE), .groups = "drop")

data_points

# 3. Des tests statistiques (au moins un sur un paramètre et une ANOVA)

Colin

# 4. Une régression et étude de corrélation

Pression et précipitations ?

Anaïs

$P_{h} - P_{atm} = \rho * g * h$

- $\rho$ masse volumique air en $kg/m^3$
- $g$ accélération de la pesanteur $9.81m/s²$
- $h$ hauteur en $m$

In [ ]:
df_regression_results = data.frame(matrix(ncol = 3, nrow = 0))
col_names <- c("(Intercept)","Pression.au.niveau.mer", "R2")
names(df_regression_results) <- col_names

for (station in sort(df_station_stats$Nom)){
    if (station!='EMBRUN' & station!="LE PUY-LOUDES"){
        # print(station)
        df_meteo_station <- df_meteo[df_meteo$Nom==station, c('ID.OMM.station', 'Date', 'Pression.au.niveau.mer', 'Variation.de.pression.en.24.heures', 'Variation.de.pression.en.3.heures', 'Précipitations.dans.la.dernière.heure', 'Précipitations.dans.les.3.dernières.heures', 'Précipitations.dans.les.24.dernières.heures', 'Type.de.tendance.barométrique', 'Point.de.rosée', 'Humidité', 'Pression.station', 'Niveau.barométrique','Altitude')]
        model_pression <- lm(Pression.station ~ Pression.au.niveau.mer, data=df_meteo_station)
        # cat("R² : ",summary(model_pression)$r.squared)

        par(bg='white')
        plot(x=df_meteo_station$Pression.au.niveau.mer, y=df_meteo_station$Pression.station, main=station)
        abline(model_pression)

        df_station_results<-data.frame(model_pression$coefficients['(Intercept)'], model_pression$coefficients['Pression.au.niveau.mer'], summary(model_pression)$r.squared,row.names = station)
        names(df_station_results)<- col_names

        df_regression_results <- rbind(df_regression_results, df_station_results)}
    }

df_regression_results

On veut tester si les stations ont les mêmes moyennes pour les variables quantitatives telles que le niveau de précipitation.

Une ANOVA requiert l'indépendance entre les variables aléatoires et que ces variables aléatoires soient suivent des lois normales de même variance. On va donc considérer les précipitations dans les 3 dernières heures car nos mesures sont échantillonnées au pas 3 heures : on va faire l'hypothèse que chaque mesure sera indépendante des autres (pas de recouvrement des mesures, mais le fait qu'on ait une série temporelle peut remettre en cause cette supposée indépendance).

In [ ]:
boxplot(Vitesse.du.vent.moyen.10.mn ~ Nom, data = df_meteo)

In [ ]:
cat("ANOVA Vitesse du vent moyen 10min\n")
aov_vitesse_vent_10mn = aov(Vitesse.du.vent.moyen.10.mn ~ Nom, df_meteo)
print(aov_vitesse_vent_10mn)
summary(aov_vitesse_vent_10mn)

Le test statistique de l'ANOVA (test de Fisher) donne une valeur de 14044 ce qui correspond à une p-value inférieure à $2e-16$. Pour un $\alpha=0.001$, on rejette l'hypothèse $H_0$ qui dit que toutes les moyennes des mesures de vitesse de vent par groupe de station sont égales (intervalle de confiance à 99,9%).

# 5. Une ACP et une PLS


**Introduction :** On souhaite réaliser une analyse en composantes principales. Pour cela, on sélectionne un sous-ensemble du jeu de données météo total. 

In [ ]:
sub <- df_meteo[df_meteo$Date > "2021-01-01 00:00:00 UTC" & df_meteo$Date <= "2021-12-31 00:00:00 UTC" & df_meteo$Nom == "TOULOUSE-BLAGNAC", ]

On sélectionne toutes les données météo de la station de Mont-de-Marsan sur l'année 2021. Commençons par analyser le jeu de données sélectionné en regardant le nombre de colonnes avec des valeurs numériques et celles qui contiennent suffisamment de valeurs pour une analyse. 

In [ ]:
for (col in colnames(sub)){
    if (!is.numeric(sub[[col]])){
        sub[[col]] <- NULL
    }
    na_frac <- sum(is.na(sub[[col]])) / nrow(sub)
    if (na_frac > 0.2) {
        sub[[col]] <- NULL
    }
}

Puis, on supprimme les colonnes inintéressantes pour la suite.

In [ ]:
colonnes_a_supprimer <- c("Altitude","EPCI..code.","region..code.","mois_de_l_annee","Latitude","Longitude", "ID.OMM.station")

for (col in colonnes_a_supprimer){
    sub[[col]]<- NULL
}


**Analyse en composantes principales**

Nous souhaitons faire la PCA sur quelques variables du jeu de données précédent. Nous prenons un nombre restreint de variables autour de quelques indicateurs importants pour les prédictions météo (température, pression, humidité, point de rosée). L'objectif est de distinguer quelles variables ont le plus d'impact sur la variance des données, ainsi que des potentielles corrélations entre elles.

In [ ]:
vars_th <- c("Température","Point.de.rosée","Humidité", "Pression.station")

sub_th <- sub[, vars_th]

# Nettoyage
n_before <- nrow(sub_th)
sub_th <- na.omit(sub_th)
n_after  <- nrow(sub_th)
cat("Lignes (avant -> après) suppression NA :", n_before, "->", n_after, "\n")
if(nrow(sub_th) < 3) stop("Trop peu de lignes après nettoyage pour faire une ACP utile.")

# ACP
library(FactoMineR)
res_pca_th <- PCA(sub_th, scale.unit = TRUE, graph = FALSE)

print(res_pca_th$eig)

ind <- res_pca_th$ind$coord[, 1:2, drop = FALSE]   # coordonnées individus
var <- res_pca_th$var$coord[, 1:2, drop = FALSE]   # coordonnées variables

# ---- 1) Nuage de points + flèches ----
max_ind <- max(abs(ind))
max_var <- max(abs(var))
sc <- ifelse(max_var == 0, 1, 0.8 * max_ind / max_var)

var_len <- sqrt(var[,1]^2 + var[,2]^2)
keep_var <- var_len > 1e-8

xlim <- range(ind[,1], var[keep_var,1] * sc) * 1.1
ylim <- range(ind[,2], var[keep_var,2] * sc) * 1.1



plot(ind, pch = 19, col = "grey40", cex = 0.6,
     xlab = paste0("PC1 (", round(res_pca_th$eig[1,2], 1), "%)"),
     ylab = paste0("PC2 (", round(res_pca_th$eig[2,2], 1), "%)"),
     xlim = xlim, ylim = ylim,
     main = "Biplot : individus + variables")
abline(h = 0, v = 0, col = "grey80")

if(any(keep_var)){
  arrows(0, 0, var[keep_var,1] * sc, var[keep_var,2] * sc,
         length = 0.08, col = "red", lwd = 1.2)
  text(var[keep_var,1] * sc * 1.06, var[keep_var,2] * sc * 1.06,
       labels = rownames(var)[keep_var], col = "red", cex = 0.9)
}

theta <- seq(0, 2*pi, length = 200)
plot(cos(theta), sin(theta), type = "l", lwd = 1.5, col = "blue",
     xlab = paste0("PC1 (", round(res_pca_th$eig[1,2], 1), "%)"),
     ylab = paste0("PC2 (", round(res_pca_th$eig[2,2], 1), "%)"),
     asp = 1, main = "Cercle des corrélations")
abline(h = 0, v = 0, col = "grey80")

arrows(0, 0, var[keep_var,1], var[keep_var,2],
       length = 0.08, col = "red", lwd = 1.2)
text(var[keep_var,1] * 1.08, var[keep_var,2] * 1.08,
     labels = rownames(var)[keep_var], col = "red", cex = 0.9)


**Conclusion**

On peut distinguer deux catégories de variables : les variables qui se projettent le plus sur la première composante principale (température et point de rosée), et celles qui se projettent plus selon la seconde (humidité et pression mesurée à la station).


**Régression PLS**

On souhaite analyser quelles variables, parmi celles du jeu de données initial, seront le plus corrélées avec les précipitation dans les dernières 24h, et trouver un modèle permettant de retrouver les précipitations dans les dernières 24h à partir de nos variables. On enlève du jeu de données les variables de précipitations autres que celles dans les dernières 24H dans le but d'expliquer les précipitations par d'autres variables. 

In [ ]:
library(pls)
# install.packages("ggplot2")
library(ggplot2)


colonnes_non_desirables <- c("Type.de.tendance.barométrique","Précipitations.dans.les.12.dernières.heures","Précipitations.dans.les.3.dernières.heures", "Précipitations.dans.les.6.dernières.heures", "Précipitations.dans.la.dernière.heure")
for (col in colonnes_non_desirables){
    sub[[col]] <- NULL
}

num_cols <- sapply(sub, is.numeric)

Y_col <- "Précipitations.dans.les.24.dernières.heures"
if(!Y_col %in% colnames(sub)) stop("La colonne précipitations n'existe pas")

X <- sub[, num_cols & colnames(sub) != Y_col]
Y <- sub[[Y_col]]

na_frac <- colSums(is.na(X))/nrow(X)
keep_cols <- names(na_frac)[na_frac <= 0.2]
X <- X[, keep_cols, drop = FALSE]
cat("Colonnes conservées :", paste(colnames(X), collapse=", "), "\n")


data_pls <- na.omit(data.frame(X, Y))
X <- data_pls[, colnames(X)]
vars_var <- sapply(X, function(x) var(x, na.rm = TRUE))
X <- X[, vars_var > 0, drop = FALSE]

Y <- data_pls$Y

cat("Nombre d'observations utilisées :", nrow(data_pls), "\n")

pls_model <- plsr(Y ~ ., data = data.frame(X, Y), scale = TRUE, validation = "CV", segments = 10)

summary(pls_model)
loadings(pls_model)

RMSEP_vals <- RMSEP(pls_model)$val[1,,]
ncomp_opt <- which.min(RMSEP_vals)
cat("Nombre optimal de composantes :", ncomp_opt, "\n")


Y_pred <- as.numeric(predict(pls_model, ncomp = ncomp_opt))

ggplot(data.frame(Observé = Y, Prédit = Y_pred),
       aes(x = Observé, y = Prédit)) +
  geom_point(color = "blue", alpha = 0.6) +
  geom_abline(slope = 1, intercept = 0, color = "red", linetype = "dashed") +
  theme_minimal() +
  labs(title = paste("PLS Observé vs Prévu - ncomp =", ncomp_opt),
       x = "Précipitations observées",
       y = "Précipitations prédites")


R2 <- 1 - sum((Y - Y_pred)^2)/sum((Y - mean(Y))^2)
cat("R² global :", round(R2*100,2), "%\n")



La régression obtenue n'est pas satisfaisante. Seulement 22% de la variabilité des données de précipitations des 24 dernières heures est capturée par le modèle. Cela nous indique que d'autres paramètres sont certainement à prendre en compte, et que le modèle permettant d'expliquer au mieux les précipitations est plus complexe que celui qu'on a essayé de trouver.

# 6. Une méthode de clustering

Nous souhaitons effectuer un clustering sur les stations météos. Pour cela, nous créons un data frame qui contient une ligne par station météo de France métropolitaine. Dans cette data frame, nous allons mettre les valeurs moyennes de température, humidité, pression, direction et vitesse moyenne du vent par mois entre janvier 2016 et décembre 2024, pour chaque station.

In [ ]:
data_stations <- data.frame(
  id = df_meteo$ID.OMM.station,
  lon = df_meteo$Longitude,
  lat = df_meteo$Latitude,
  Ville = df_meteo$Nom,
  date = df_meteo$Date,
  temp = df_meteo$Température,
  hum = df_meteo$`Humidité`,
  pression = df_meteo$Pression.station,
  dir_vent = df_meteo$Direction.du.vent.moyen.10.mn,
  vit_vent = df_meteo$Vitesse.du.vent.moyen.10.mn,
  mois = df_meteo$mois_de_l_annee
)

data_stations_metropole <- data_stations[(data_stations$lat<=52)&(data_stations$lat>=42)&(data_stations$lon<=10)&(data_stations$lon>=-10),]

coord_stations <- data_stations_metropole %>%
  select(id, lon, lat, Ville) %>%
  distinct(id, .keep_all = TRUE)

#head(data_stations_metropole)
mois_par_trimestre <- matrix(1:12, ncol=4, byrow = TRUE)
df_bystations <- NULL

#for (annee_encours in 2016:2024){
annee_encours <- 2017

  for(trimestre_encours in 1:3){
# Filtrer et calculer la moyenne de température par station
    moyennes_par_station <- data_stations_metropole %>%
      filter(format(date,"%Y")==annee_encours, mois %in% as.vector(mois_par_trimestre[trimestre_encours, ])) %>%
      group_by(id) %>%
      summarise(temp_moyenne = mean(temp, na.rm = TRUE),
                hum_moyenne = mean(hum, na.rm = TRUE),
                pression_moyenne = mean(pression, na.rm = TRUE)) %>%
      ungroup()
      # Création du nom de la colonne
    nom_colonne_temp <- paste0("temp_T", trimestre_encours, "_", annee_encours)
    nom_colonne_hum <- paste0("hum_T", trimestre_encours, "_", annee_encours)
    nom_colonne_pression <- paste0("pression_T", trimestre_encours, "_", annee_encours)
    # Renommer la colonne de température moyenne
    colnames(moyennes_par_station)[colnames(moyennes_par_station) == "temp_moyenne"] <- nom_colonne_temp
    colnames(moyennes_par_station)[colnames(moyennes_par_station) == "hum_moyenne"] <- nom_colonne_hum
    colnames(moyennes_par_station)[colnames(moyennes_par_station) == "pression_moyenne"] <- nom_colonne_pression
    # Fusion avec le dataframe final
    if (is.null(df_bystations)) {
      df_bystations <- moyennes_par_station
    } else {
      df_bystations <- full_join(df_bystations, moyennes_par_station, by = "id")
    }

}
#}
# Afficher
print(head(df_bystations))

kmeans_stations <- kmeans(df_bystations, centers=4)
cat("Groupes des stations pour le premier K-means :\n")
print(kmeans_stations$cluster)

# AJOUT : Ajouter les clusters comme colonne
df_bystations$groupe <- kmeans_stations$cluster
# AJOUT : Joindre les coordonnées
df_bystations_geo <- dplyr::left_join(df_bystations, coord_stations, by = "id")
# AJOUT : Palette de couleurs
library(RColorBrewer)
couleurs <- brewer.pal(4, "Set1")
4:45
# Affichage sur la carte
ggplot() +
  geom_polygon(data = france, aes(x = long, y = lat, group = group),
               fill = "lightgray", color = "white") +
  geom_point(data = df_bystations_geo, aes(x = lon, y = lat, color = factor(groupe)), size = 3) +
  scale_color_manual(values = couleurs) +
  theme_minimal() +
  coord_fixed(ratio = 1.2) +
  ggtitle("Carte des stations météo de France métropolitaine selon le clustering")